In [2]:
# set up objects and necessary libs
import csv
import copy
from pprint import pprint as pprint
import statistics

participants = {}
# need to look into these ppl is all
blacklisted  = []

In [3]:
# read in the participants from the session
with open('csvs/roster.csv', mode='r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    roster_count = 0
    for row in csv_reader:
        if row['pid'] not in blacklisted:
            participants[row['pid']] = {}
            participants[row['pid']]['name'] = row['name']
            roster_count += 1
            
print("Read in " + str(roster_count) + " participants from this session.")
#pprint(participants)

Read in 79 participants from this session.


In [4]:
# set up the participants object where each participant is mapped to each other participant
# the mapping value will be a list of size 4
# first index will represent whether those two participants interacted or not during ProtoTeams
# second index will represent the rating of the interaction that occurred during ProtoTeams 
# third index will represent the degree of closeness of their social tie prior to ProtoTeams
for participant in participants:
    for other_participant in participants:
        if participant != other_participant:
            participants[participant][other_participant] = [0,0]

#pprint(participants)

In [5]:
# get the interactions data from the app
with open('csvs/interactions_84qls.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    roster_count = 0
    interactions = 0
    for row in csv_reader:
        if (row["people_interacted_with"][1:-1] != "blacklisted"):
            worked_with = row["people_interacted_with"][1:-1].replace(";","").split(" ")
            worked_with_scores = row["all_interactions"][1:-1].split(" ")[1::2]
            if row["participant"] not in participants:
                print(f"{row['participant']}")
            else:
                roster_count += 1
            for groupmate_num, groupmate in enumerate(worked_with):
                if row["participant"] not in participants:
                    continue
                # mark them as interacted
                participants[row['participant']]['interacted'] = 1
                if groupmate in participants[row["participant"]]:
                    participants[row["participant"]][groupmate] = [1,float(worked_with_scores[groupmate_num][:-1])]
                    print(participants[row["participant"]][groupmate])
                    interactions += 1
        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Updated {roster_count} pids.')
    print(f'Tracked {interactions} interactions.')
    
#pprint(participants)

[1, 2.5]
[1, 0.0]
[1, 2.5]
[1, 2.0]
[1, 1.0]
[1, 2.5]
[1, 3.0]
[1, 2.0]
[1, 3.5]
[1, 3.0]
[1, 3.0]
[1, 0.0]
[1, 2.5]
[1, 3.0]
[1, 2.5]
[1, 0.0]
[1, 0.0]
[1, 2.0]
[1, 3.0]
[1, 0.5]
[1, 3.0]
[1, 3.0]
[1, 2.5]
[1, 3.0]
[1, 2.5]
[1, 2.5]
[1, 3.0]
[1, 2.0]
[1, 4.0]
[1, 4.0]
[1, 3.5]
[1, 3.0]
[1, 3.0]
[1, 2.5]
[1, 3.0]
[1, 0.0]
[1, 0.0]
[1, 3.0]
[1, 2.5]
[1, 3.5]
[1, 2.5]
[1, 2.5]
[1, 0.0]
[1, 0.0]
[1, 1.5]
[1, 2.0]
[1, 3.5]
[1, 1.0]
[1, 2.0]
[1, 2.0]
[1, 2.5]
[1, 2.5]
[1, 2.5]
[1, 0.0]
[1, 2.0]
[1, 2.5]
[1, 3.0]
[1, 3.0]
[1, 3.0]
[1, 2.0]
[1, 0.0]
[1, 0.5]
[1, 4.0]
[1, 4.0]
[1, 2.5]
[1, 2.5]
[1, 3.5]
[1, 3.5]
[1, 1.5]
[1, 0.0]
[1, 2.5]
[1, 3.5]
[1, 0.0]
[1, 3.0]
[1, 2.5]
[1, 3.5]
[1, 4.0]
[1, 0.0]
[1, 0.0]
[1, 3.5]
[1, 0.0]
[1, 3.5]
[1, 3.0]
[1, 3.5]
[1, 3.5]
[1, 2.5]
[1, 0.0]
[1, 4.0]
[1, 3.0]
[1, 0.0]
[1, 3.0]
[1, 3.0]
[1, 2.5]
[1, 3.5]
[1, 3.0]
[1, 3.5]
[1, 0.0]
[1, 3.0]
[1, 3.5]
[1, 1.0]
[1, 2.5]
[1, 2.5]
[1, 3.0]
[1, 3.5]
[1, 3.5]
[1, 3.0]
[1, 0.0]
[1, 3.0]
[1, 2.5]
[1, 3.0]
[1, 3.0]
[

In [6]:
#process the social_ties given by the survey results
nclose = 0
bclose = 0
mclose = 0
close = 0
vclose = 0
noresponse = 0

def getValueFromCloseness(closeness):
    global nclose, bclose, mclose, close, vclose, noresponse
    if closeness == "I don't know them":
        nclose += 1
        return 0
    elif closeness == "I barely know them":
        bclose += 1
        return 1
    elif closeness == "I know them":
        mclose += 1
        return 2
    elif closeness == "We are close":
        close += 1
        return 3
    elif closeness == "We are very close":
        vclose += 1
        return 4
    else:
        noresponse += 1
        return None
    
with open('csvs/social_ties.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    social_ties = 0
    waitlisted_count = 0
    waitlisted_people = []
    for row in csv_reader:
        # catch people who aren't in participants (i think there is one)
        if row['PID'] not in participants:
            print("PID found in survey but not found in participants: " + row['PID'])
        # only check for people that are will be in the datasheet
        if row['PID'] in participants:
            curr_pid = row['PID']
            curr_name = participants[curr_pid]['name']
            # get the ratings for the people they will have dyads with
            for participant in participants:
                name = participants[participant]['name']
                if participant != row['PID']:
                    if (name in row):
                        participants[curr_pid][participant].append(getValueFromCloseness(row[name]))
                        social_ties += 1
                    else:
                        # 0 rating for waitlisted people for now
                        participants[curr_pid][participant].append(0)

        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Found and counted {nclose + bclose + mclose + close + vclose} out of {social_ties} social tie ratings.')
    print(f'There were {noresponse} social tie ratings not given.')
    print(f'{nclose} ties were considered not close.')
    print(f'{bclose} ties were considered barely close.')
    print(f'{mclose} ties said they knew each other.')
    print(f'{close} ties were considered close.')
    print(f'{vclose} ties were considered very close.')
    
#pprint(participants)

PID found in survey but not found in participants: a15723106
Processed 76 lines.
Found and counted 5616 out of 5850 social tie ratings.
There were 234 social tie ratings not given.
5278 ties were considered not close.
153 ties were considered barely close.
97 ties said they knew each other.
36 ties were considered close.
52 ties were considered very close.


In [7]:
# get the demographic, social data
with open('csvs/alison_coded_pre_survey.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    response_count = 0
    feature_count = 0
    invalid_response_count = 0
    for row in csv_reader:
        if (row['PID'] in participants):
            
            # demographic data
            participants[row['PID']]['age'] = row['Age']
            participants[row['PID']]['gender'] = row['Gender']
            participants[row['PID']]['nationality'] = row['Nationality']        
            if row['School Year'] == '1st year (Freshman)':
                participants[row['PID']]['school_year'] = 1
            elif row['School Year'] == '2nd year (Sophomore)':
                participants[row['PID']]['school_year'] = 2
            elif row['School Year'] == '3rd year (Junior)':
                participants[row['PID']]['school_year'] = 3
            elif row['School Year'] == '4th year (Senior)':
                participants[row['PID']]['school_year'] = 4
            elif row['School Year'] == '5th year or above':
                participants[row['PID']]['school_year'] = 5
            else:
                participants[row['PID']]['school_year'] = 6
           
            # individual characteristics
            participants[row['PID']]['social_skills'] = row['Social Skills']
            participants[row['PID']]['leadership'] = row['Leadership']
            participants[row['PID']]['intercult_sens'] = row['Intercultural Sensitivity']
       
            feature_count += 7
            response_count += 1
        else:
            invalid_response_count += 1
        line_count += 1
    print(f'Processed {line_count} lines.')
    print(f'Updated the demographic data for {response_count} people.')
    print(f'Produced {feature_count} features.')
    print(f'{invalid_response_count} people will not have their survey responses used.')

Processed 77 lines.
Updated the demographic data for 73 people.
Produced 511 features.
4 people will not have their survey responses used.


In [8]:
# get the people who actually ended up teaming together 
with open('csvs/alison_final_teams_exclude.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    team_count = 0
    final_teammates = 0
    for row in csv_reader:
        team_count += 1
        for i in range(1,6):
            curr_stud = row['Member '+ str(i)] 
            if curr_stud in participants:
                for j in range(1, 6):
                    other_stud = row['Member '+ str(j)]
                    if j != i and other_stud in participants:
                        if 'final_team' not in participants[curr_stud]:
                            participants[curr_stud]['final_team'] = [other_stud]
                        else:
                            participants[curr_stud]['final_team'].append(other_stud)
                        final_teammates += 1
        line_count += 1
                        
    print(f'Processed {line_count} lines.')
    print(f'{final_teammates} final teammates were tracked.')
    
#pprint(participants)
            

Processed 13 lines.
260 final teammates were tracked.


In [9]:
# prune participants to only include people who participated in the field study and completed the survey
orig_participants = copy.deepcopy(participants)
features = ['name','age','gender','nationality','school_year','social_skills',
            'leadership','intercult_sens','interacted','final_team']

for removee in orig_participants:
    # account for when we've already remove people
    if removee not in participants:
        continue
    for feature in features:
        # remove all info on this participant if they're missing a feature
        if feature not in participants[removee]:
            for keeper in participants:
                participants[keeper].pop(removee, None)
            participants.pop(removee, None)
            break
            
print(f'There are {len(participants)} valid data points left.')
pprint(participants.keys())
#pprint(participants)

There are 53 valid data points left.
dict_keys(['a15071063', 'a15741122', 'a15744874', 'a15741940', 'a15741190', 'a15695172', 'a13486546', 'a12999463', 'a15741587', 'a14884278', 'a15740800', 'a13355516', 'a15757631', 'a14791048', 'a15743022', 'a161113555', 'a13586094', 'a15700334', 'a14016211', 'a15744119', 'a12685874', 'a14033703', 'a13445041', 'a13516281', 'a13053427', 'a15712849', 'a16140608', 'a15737360', 'a15350429', 'a15742782', 'a15757849', 'a15650515', 'a15151514', 'a13848955', 'a13630136', 'a15760006', 'a16140372', 'a14707793', 'a14696531', 'a15757861', 'a13925466', 'a15740484', 'a15757893', 'a15741910', 'a15741283', 'a14484035', 'a15703188', 'a13691086', 'a13598112', 'a15741110', 'a15740470', 'a15678959', 'a13328999'])


In [10]:
# dyad maker and dyad tracker
class dyad:
    
    # construct the dyad opbject
    def __init__(self, personA, personB):
        
        # the name for this dyad
        self.dyad = personA + '-' + personB
        
        # calculated based on the average of how well the two people knew each other
        self.social_tie = ('1' if participants[personA][personB][2] > 0 or participants[personB][personA][2] > 0 else '0')
        self.mutual = ('1' if participants[personA][personB][2] > 0 and participants[personB][personA][2] > 0 else '0')
        self.social_tie_avg = str((participants[personA][personB][2] + participants[personB][personA][2]) / 2)

        # received from the interaction data
        self.interacted = ('0' if participants[personA][personB][0] == 0 else '1')
        self.avg_rating = str(participants[personA][personB][1] + participants[personB][personA][1] / 2)
        
        # these features come from the pre-survey
        
        # The value for these feature is either 0/1 for same/different or the difference between the two people
        self.age = str(abs(int(participants[personA]['age']) - int(participants[personB]['age'])))
        self.gender = ('1' if participants[personA]['gender'] == participants[personB]['gender'] else '0')
        self.both_male = ('1' if participants[personA]['gender'] == 'Man' 
                              and participants[personB]['gender'] == 'Man' else '0')
        self.both_female = ('1' if participants[personA]['gender'] == 'Woman' 
                              and participants[personB]['gender'] == 'Woman' else '0')
        self.nationality = ('1' if participants[personA]['nationality'] == 
                                    participants[personB]['nationality'] else '0')
        self.school_year = str(abs(int(participants[personA]['school_year']) - 
                                   int(participants[personB]['school_year'])))
        
        # individual characteristics, value is the difference between each person
        self.social_skills = str(abs(int(participants[personA]['social_skills']) - 
                                     int(participants[personB]['social_skills'])))
        self.leadership = str(abs(int(participants[personA]['leadership']) - 
                                     int(participants[personB]['leadership'])))
        self.intercult_sens = str(abs(int(participants[personA]['intercult_sens']) - 
                                     int(participants[personB]['intercult_sens'])))
        
        # 0/1 depending on whether or not these two ended up on the same team, aka, the truth-value for our dyad
        self.team = ('1' if personB in participants[personA]['final_team'] else '0')
        
    # turn our dyad object into a string
    def toString(self):
        return ({'dyad': self.dyad,'social_tie': self.social_tie,'mutual': self.mutual,
                 'social_tie_avg': self.social_tie_avg,'interacted': self.interacted,'avg_rating': self.avg_rating,
                 'age': self.age, 'gender': self.gender, 'both_male': self.both_male,
                 'both_female': self.both_female, 'nationality': self.nationality,'school_year': self.school_year,
                 'social_skills': self.social_skills, 'leadership': self.leadership,
                 'intercultural_sensitivity': self.intercult_sens, 'team': self.team})

In [11]:
# build the master spreadsheet of dyad data
header = ['dyad','social_tie','mutual','social_tie_avg','interacted','avg_rating']
header += ['age','gender','both_male','both_female','nationality','school_year'] 
header += ['social_skills','leadership','intercultural_sensitivity']
header += ['team']

dyad_count = 0
dyads = {}

with open('csvs/alison_dyad_data_excluded.csv', mode='w') as dyad_file:
    
    # set up the spreadsheet to be ready for writing and with it's header
    dyad_writer = csv.DictWriter(dyad_file, fieldnames=header)
    dyad_writer.writeheader()
    
    # produce each dyad
    for participant in participants:
        for other_participant in participants[participant]:
             if (participant != other_participant and 
                 (other_participant + '-' + participant) not in dyads and
                 other_participant not in features):
                new_dyad = dyad(participant, other_participant)
                dyad_writer.writerow(new_dyad.toString())
                dyads[new_dyad.dyad] = new_dyad
                dyad_count += 1

#pprint(dyads)    
print(f'Completed {dyad_count} dyads.')

Completed 1378 dyads.


In [12]:
# find some percentages of teams formed based on social_tie, interactions, etc.
interacted_dyad_count = 0
st_dyad_count = 0
teamed_dyad_count = 0
st_interacted_dyad_count = 0
teamed_interacted_dyad_count = 0
teamed_st_dyad_count = 0
teamed_interacted_st_dyad_count = 0

for dyad in dyads:
    # interacted
    if (dyads[dyad].interacted == '1'):
        interacted_dyad_count += 1
    # social ties
    if (dyads[dyad].social_tie == '1'):
        st_dyad_count += 1
        if (dyads[dyad].interacted == '1'):
            st_interacted_dyad_count += 1
    # teamed
    if (dyads[dyad].team == '1'):
        teamed_dyad_count += 1
        if (dyads[dyad].interacted == '1' and dyads[dyad].social_tie == '1'):
            teamed_interacted_st_dyad_count += 1
        if (dyads[dyad].interacted == '1'):
            teamed_interacted_dyad_count += 1
        if (dyads[dyad].social_tie == '1'):
            teamed_st_dyad_count += 1

print(f'{interacted_dyad_count} dyads interacted during ProtoTeams.')
print(f'{st_dyad_count} previous social ties.')           
print(f'{st_interacted_dyad_count} dyads with prior social ties also interacted during ProtoTeams.')
# percentage of interactions between people who already knew each other, to some degree.
print(f'{round(st_interacted_dyad_count / st_dyad_count * 100, 3)}% of dyads who with prior social ties also interacted during ProtoTeams.\n')

print(f'{teamed_dyad_count} dyads ended up in teams.')
print(f'{teamed_interacted_dyad_count} of dyads in teams interacted during ProtoTeams.')
print(f'{teamed_st_dyad_count} of dyads in teams had previous social ties.')
print(f'{teamed_interacted_st_dyad_count} of dyads in teams who had previous social ties and interacted in ProtoTeams.\n')

# percentage of people who ended up on a team together and interacted during prototeams
print(f'{round(teamed_interacted_dyad_count / teamed_dyad_count * 100, 3)}% of dyads in teams who interacted during prototeams')
# percentage of people who ended up on a team together and knew each other beforehand
print(f'{round(teamed_st_dyad_count / teamed_dyad_count * 100, 3)}% of dyads in teams who had prior social ties')
# percentage of people who ended up on a team together and knew each other beforehand and interacted during prototeams
print(f'{round(teamed_interacted_st_dyad_count / teamed_dyad_count * 100, 3)}% of dyads in teams who interacted during prototeams and had prior social ties')
# percentage of people in teams together who knew each other beforehand and also interacted during prototeams
# out of the people in teams who knew each other beforehand
print(f'{round(teamed_interacted_st_dyad_count / teamed_st_dyad_count * 100, 3)}% of dyads in teams who had prior social ties also interacted.')
# percentage of people in teams together who knew each other beforehand and also interacted during prototeams
# out of the people in teams who interacted during prototeams
print(f'{round(teamed_interacted_st_dyad_count / teamed_interacted_dyad_count * 100, 3)}% of dyads in teams who interacted also had social ties.')


248 dyads interacted during ProtoTeams.
148 previous social ties.
19 dyads with prior social ties also interacted during ProtoTeams.
12.838% of dyads who with prior social ties also interacted during ProtoTeams.

93 dyads ended up in teams.
21 of dyads in teams interacted during ProtoTeams.
33 of dyads in teams had previous social ties.
7 of dyads in teams who had previous social ties and interacted in ProtoTeams.

22.581% of dyads in teams who interacted during prototeams
35.484% of dyads in teams who had prior social ties
7.527% of dyads in teams who interacted during prototeams and had prior social ties
21.212% of dyads in teams who had prior social ties also interacted.
33.333% of dyads in teams who interacted also had social ties.


In [13]:
# diversity metrics
def find_gender_diversity(members):
    categories = {'Man': 0, 'Woman': 0}
    num_mems = len(members)
    for member in members:
        if participants[member]['gender'] == 'Man':
            categories['Man'] += 1
        if participants[member]['gender'] == 'Woman':
            categories['Woman'] += 1
    # blau's index
    gender_sum = 0
    if num_mems != 0:
        for category in categories:
            gender_sum += (categories[category]*(categories[category]))/(num_mems*(num_mems-1))
    #print(1 - gender_sum)
    # just return the counts
    return categories['Man'], categories['Woman']


In [14]:
# build a team facts datasheet
# TODO
with open('csvs/alison_final_teams_exclude.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    team_count = 0
    proto_count = 0
    people_count = 0
    fieldnames = ['Team','Num_Members','Num_PT_Participants', 'Num_Dyads','Num_ProtoTeams_Interactions','Num_Social_Ties',
                  'Average_Social_Tie_Rating','Num_Male','Num_Female','Age_Variance']
    # col for prior interactions Score! age spread, social skills
    with open('csvs/teams_info.csv', mode='w') as csv_file2:
        teams_writer = csv.DictWriter(csv_file2, fieldnames=fieldnames)                                    
        teams_writer.writeheader()
        
              
        # build a age-to-team dataset
        with open('csvs/ages_and_teams.csv', mode='w') as csv_file3:
            age_writer = csv.DictWriter(csv_file3, fieldnames=['Person','Age','Team'])                                    
            age_writer.writeheader()

            for row in csv_reader:
                # get the members of this team
                team_count += 1
                curr_team = row['Teams']
                members = []
                proto_members = []
                for i in range(1,6):
                    if row['Member '+ str(i)] != '':
                        curr_stud = row['Member '+ str(i)] 
                        members.append(curr_stud)
                        people_count += 1
                        if curr_stud in participants:
                            proto_count += 1
                            proto_members.append(curr_stud)
                        
                # things we are interested in  
                dyad_count = 0
                interactions_sum = 0
                interacted_people = []
                prior_social_ties = 0
                prior_people = []
                avg_prior_rating = 0
                ages = []
                social_skill_scores = []
                
                # loop through the team members to get some important info
                for person in proto_members:
                    person_age = int(participants[person]['age'])
                    age_writer.writerow({'Person': person, 'Age': person_age, 'Team': curr_team})
                    ages.append(person_age)
                    social_skill_scores.append(int(participants[person]['social_skills']))
                    for other_person in proto_members:
                        if (person + '-' + other_person) in dyads:
                            dyad_count += 1
                            curr_dyad = dyads[(person + '-' + other_person)]
                            if int(curr_dyad.interacted) == 1:
                                interactions_sum += 1
                                interacted_people += [[person,other_person]]
                            if int(curr_dyad.social_tie) == 1:
                                prior_social_ties += 1
                                prior_people += [[person,other_person]]
                                avg_prior_rating += float(curr_dyad.social_tie_avg)
                    
                if (prior_social_ties > 0):
                    avg_prior_rating = avg_prior_rating / prior_social_ties 
                    
                # number of men, women, and non-binary
                man_count, woman_count = find_gender_diversity(proto_members)
                
                # coefficient of variance for ages
                if (ages != []):
                    age_var = round(statistics.stdev(ages) / statistics.mean(ages), 3)
                
                # avg social skills score
                if (social_skill_scores != []):
                    ss_avg = round(statistics.mean(social_skill_scores), 3)
                
                # write our to team info csv file
                teams_writer.writerow({'Team': team_count, 'Num_Members': len(members), 
                                       'Num_PT_Participants': len(proto_members),'Num_Dyads': dyad_count, 
                                       'Num_ProtoTeams_Interactions': interactions_sum,
                                       'Num_Social_Ties': prior_social_ties,
                                       'Average_Social_Tie_Rating': avg_prior_rating,'Num_Male': man_count,
                                       'Num_Female': woman_count,'Age_Variance': age_var,})


    print(f'{team_count} teams were tracked.')
    print(f'There are {people_count} people overall.')
    print(f'{proto_count} of them participated in ProtoTeams.')
    
#pprint(participants)
            

13 teams were tracked.
There are 65 people overall.
53 of them participated in ProtoTeams.
